## 自动生成聊天（第一句）的尝试

这个脚本由李鲁鲁开发， 属于[Chat凉宫春日](https://github.com/LC1332/Chat-Haruhi-Suzumiya) 

用于研究是否能够基于GPT生成大量的对话数据

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 由李鲁鲁，冷子昂，闫晨曦，封小洋等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。

冷子昂参与了早期Gradio的开发，并且参与了后端和前端的选型

闫晨曦将李鲁鲁的notebook重构为app.py

封小洋进行了中文转日文模型的选型

</details>

这个脚本主要是利用-1脚本已经抽取的关键词，两个jsonl文件

来进行工作

- [x] 组织句子生成的prompt
- [x] 引入故事中的关键词
- [x] 检验返回的jsonl是否合理
- [x] 将生成结果 存储到google drive
- [x] 合成一个大的jsonl


In [ ]:
#@title 安装环境
! pip -q install openai gradio transformers tiktoken langchain gradio

## 输入OpenAI Token

In [ ]:
import os
import openai

openai.api_key = 'sk-lfrdoJKjlG' # 在这里输入你的OpenAI API Token
os.environ["OPENAI_API_KEY"] = openai.api_key 

<details>
  <summary> 让GPT老师写一个jsonl读取 </summary>

  实现一个python函数，输入一个文件名，是一个jsonl文件
  每一行是一个json（会包含中文）
  将这个jsonl解析到一个list并返回

</details>

In [ ]:
#@title 实现parse_jsonl_file函数

import json

def parse_jsonl_file(file_name):
    json_list = []
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in f:
            json_obj = json.loads(line.strip())
            json_list.append(json_obj)
    return json_list

In [ ]:
fname_story_data = '/content/all_story_keywords.jsonl'

story_keywords = parse_jsonl_file(fname_story_data )

fname_chat_data = '/content/all_chat_datas.jsonl'

chat_datas = parse_jsonl_file(fname_chat_data)

In [ ]:
#@title 统一story和chat中的keywords字段（前者本来为entity）

def replace_entity_with_keywords(story_keywords):
    for item in story_keywords:
        if "Entity" in item:
            item["keywords"] = item.pop("Entity")
    return story_keywords

story_keywords = replace_entity_with_keywords(story_keywords)

In [ ]:
#@title 去除停词关键词

stop_words = ['春日','阿虚','凉宫','凉宫春日']


def remove_stop_words(data, stop_words):
    stop_words_set = set(stop_words) # 转换为set方便查找
    for item in data:
        # if "keywords" in item: # 判断关键词列表是否存在
        item["keywords"] = [w for w in item["keywords"] if w not in stop_words_set]
    return data

chat_datas = remove_stop_words(chat_datas, stop_words)
story_keywords = remove_stop_words(story_keywords, stop_words)


In [ ]:
print(len(story_keywords))
print(len(chat_datas))

print(story_keywords[0])
print(chat_datas[0])

48
556
{'keywords': ['电脑', '资讯化的时代', '拍立得', '作战计划', '把握机会']}
{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」', 'keywords': ['计算机课', 'Python']}


## 让我们来测试Prompt！

In [ ]:
#@title 定义organize_samples 和 list_to_string

from typing import List, Dict, Tuple
import numpy as np

def list_to_string(lst):
    result = ''
    for item in lst:
        result += str(item) + '\n'
    return result

def organize_samples(sel_chat_datas: List[Dict[str, str]]) -> Tuple[List[Dict], List[Dict], List[str]]:
    # stop_words = ['春日', '阿虚', '凉宫', '凉宫春日']
    sample_input = []
    sample_output = []
    all_keywords = set()
    for element in sel_chat_datas:
        keywords = element['keywords']  # [kw for kw in element['keywords'] if kw not in stop_words]
        np.random.shuffle(keywords)
        sample_input.append({'keywords': keywords})
        output_element = {
            'keywords': keywords,
            'role': element['role_A'],
            'text': element['query'],
        }
        sample_output.append(output_element)
        for kw in keywords:
            all_keywords.add(kw)
    return sample_input, sample_output, list(all_keywords)

In [ ]:
#@title 测试organize_samples

sel_chat_data = [chat_datas[i] for i in range(3)]

sample_input, sample_output, _ = organize_samples(sel_chat_data)

print(list_to_string(sample_input))
# print('\n')
print(list_to_string(sample_output))

{'keywords': ['Python', '计算机课']}
{'keywords': ['什么样的程序', '写程序']}
{'keywords': ['程序', '赚很多钱', '预测彩票']}

{'keywords': ['Python', '计算机课'], 'role': '阿虚', 'text': '「今天在计算机课上老师教了我写Python!」'}
{'keywords': ['什么样的程序', '写程序'], 'role': '阿虚', 'text': '「你想写一个什么样的程序呢？」'}
{'keywords': ['程序', '赚很多钱', '预测彩票'], 'role': '阿虚', 'text': '「如果有一个能预测彩票的程序，我们岂不是能赚很多钱？」'}



## 后面的目标

我写到这里觉得有点头秃

我们要组一个one-shot或者说two-shot的prompt

这里关键是组织新的input很令人头痛

因为我希望新的input是这样的，keywords不出现任意原来sample input的列表中，

这样太耦合了，我们先做一个foo_input，然后把prompt组装的函数给写了



In [ ]:
#@title 一个之后会废弃的foo_sample和foo_input函数

import random 

n = len(chat_datas)
sel_all = random.sample(range(0, n), 20)

sel_sample = sel_all[:10]
sel_input = sel_all[10:]

def foo_sample():
    sel_chat_data = [chat_datas[i] for i in sel_sample]

    sample_input, sample_output, sample_keywords = organize_samples(sel_chat_data)

    return sample_input, sample_output, sample_keywords
    

def foo_input(sample_keywords):
    sel_chat_data = [chat_datas[i] for i in sel_input]

    sample_input, _ , _ = organize_samples(sel_chat_data)

    return sample_input

In [ ]:
TANSFER_PROMPT = """
根据keywords的内容补全text
text为对于凉宫春日剧情的一些讨论问题，role不可以是春日或者凉宫春日
role可以是阿虚、朝比奈、老师等凉宫春日中，非春日的其他角色
role也可以是任意其他动漫中的角色
用一致性的语言风格，根据每行中的json内容，根据keywords中的关键字，补全text的内容。
"""

In [ ]:
#@title 定义生成prompt

from langchain.chat_models import ChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chatModel = ChatOpenAI(temperature=0.1, max_tokens = 2000)

sample_input, sample_output, sample_keywords = foo_sample()
query_input = foo_input(sample_keywords)

def generate_with_keywords( sample_input, sample_output, query_input ):

    div_k = 4
    input1 = list_to_string( sample_input[:div_k] )
    output1 = list_to_string( sample_output[:div_k] )
    input2 = list_to_string( sample_input[div_k:] )
    output2 = list_to_string( sample_output[div_k:] )

    query = list_to_string(query_input)

    messages = [
        SystemMessage(content=TANSFER_PROMPT),
        HumanMessage(content=input1),
        AIMessage(content=output1),
        HumanMessage(content=input2),
        AIMessage(content=output2),
        HumanMessage(content=query)
    ]

    return_msg = chatModel(messages)

    return return_msg.content

In [ ]:
response = generate_with_keywords( sample_input, sample_output, query_input )
print(response)

接下来我们要开始真正组织的input

<details>
  <summary> 让GPT老师开发一下数据类 </summary>

我希望实现一个python的类 DataLoader。这个python类由一个list初始化。并且内部会记录list的个数n

初始化的时候，会自动建立一个长度为n的shuffle_id,用来从整个list随机取元素，并把currend_id赋值为0

这个类有一个方法getData()， 每次会根据current_id依次返回一个数据（序号为shuffle_id[current_id]的数据）

当返回了n个数据的时候，DataLoader会重新shuffle数据的顺序。

并且可以额外设定一个数据k(默认值为10)，当n > 2k时，每次重新shuffle会保证新的shuffe_id序列的前k-1个元素和上一次shuffe_id的后k-1个元素之间没有重复元素。这样保证在连续取k个数据时，总是不重复


</details>

In [ ]:
#@title 建立DataLoader类

import random

class DataLoader:
    def __init__(self, data, k=10):
        self.data = data
        self.n = len(data)
        self.k = k
        self.current_id = 0
        self.shuffle_id = list(range(self.n))
        random.shuffle(self.shuffle_id)
        self.previous_tail = self.shuffle_id[-self.k+1:]

    def shuffle(self):
        if self.n <= 2 * self.k:
            random.shuffle(self.shuffle_id)
        else:
            random.shuffle(self.shuffle_id)
            head = self.shuffle_id[:self.k-1]
            flag = True
            count = 0

            min_ovlp_num = 999
            min_ovlp_plan = []

            while count < 10 and flag == True:
                count = count + 1
                inverse_flag = False
                ovlp_num = 0
                for id in head:
                    if id in self.previous_tail:
                        ovlp_num = ovlp_num + 1
                        inverse_flag = True

                if ovlp_num < min_ovlp_num:
                    min_ovlp_num = ovlp_num
                    min_ovlp_plan = self.shuffle_id.copy()

                if False == inverse_flag:
                    flag = False
                    break

                random.shuffle(self.shuffle_id)
                head = self.shuffle_id[:self.k-1]

            # print('shuffle test time ', count, ' min ovlp = ', min_ovlp_num)

            if min_ovlp_num > 0:
                self.shuffle_id = min_ovlp_plan

            head = self.shuffle_id[self.k-1:]
            tail = self.shuffle_id[-self.k+1:]

            self.shuffle_id = head + self.shuffle_id[:self.k-1] + tail
            random.shuffle(self.shuffle_id)
            self.previous_tail = tail

    def get_data(self):
        if self.current_id >= self.n:
            self.shuffle()
            self.current_id = 0
        data = self.data[self.shuffle_id[self.current_id]]
        self.current_id += 1
        return data

In [ ]:
data_story = DataLoader(story_keywords, 10)

data_chat_as_story = DataLoader(chat_datas, 10)

data_chat = DataLoader(chat_datas, 10)

for i in range(900):
    data = data_story.get_data()
    if i % 300 == 0:
        print(data)

{'keywords': ['侦探推理小说迷', '推理研究会']}
{'keywords': ['长门有希', '社团教室']}
{'keywords': ['名侦探', '小众文化类同好会']}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/GPTData/Haruhi-AutoFirst/"

Mounted at /content/drive


In [ ]:
import datetime
import os

# save_path = "/content/drive/MyDrive/GPTData/Haruhi-AutoFirst/"

def save_to_file(response, save_path):
    # 获取当前时间并转换成字符串作为文件名
    now = datetime.datetime.now()
    timestamp_str = now.strftime("%Y-%m-%d-%H-%M-%S")

    # 拼接完整的文件路径
    file_path = os.path.join(save_path, f"{timestamp_str}.txt")

    # 如果文件已经存在，则在文件名尾部加上一个随机字符串
    while os.path.exists(file_path):
        random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))
        file_path = os.path.join(save_path, f"{timestamp_str}-{random_suffix}.txt")

    # 将response写入文件
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(response)

In [ ]:
from tqdm import tqdm

batch_size = 10

for iter_time in tqdm(range(700),desc='autoGenerating'):

    chat_data = []

    for _ in range(batch_size):
        chat_data.append( data_chat.get_data() )

    sample_input, sample_output, sample_keywords = organize_samples(chat_data)

    #这里我们还要组织query_input

    query_input = []

    for input in sample_input:
        target_n = len( input['keywords'] )
        target_n = max(2, target_n )

        count_time = 0
        max_len = -999
        max_len_plan = []

        while count_time < 15:
            #随机抽取一个story的keyword
            count_time = count_time + 1
            if iter_time % 2 == 0:
                story_keyword = data_story.get_data()
            else:
                story_keyword = data_chat_as_story.get_data()

            filtered_keyword = [w for w in story_keyword["keywords"] if w not in sample_keywords]
            if len(filtered_keyword) >= target_n:
                story_keyword['keywords'] = random.sample(filtered_keyword, min(target_n, len(filtered_keyword)))
                break
            else:
                if len(filtered_keyword)>max_len:
                    max_len = len(filtered_keyword)
                    # story_keyword['keywords'] = filtered_keyword
                    max_len_plan = filtered_keyword.copy()

        if len(story_keyword['keywords'] ) < target_n:
            story_keyword['keywords'] = max_len_plan
            # print('use max len plan ', target_n - len(story_keyword['keywords'] ))
        query_input.append( {'keywords':story_keyword['keywords']} )

        for keyword in story_keyword['keywords']:
            sample_keywords.append(keyword)

    # response = generate_with_keywords( sample_input, sample_output, query_input )
    try:
        response = generate_with_keywords(sample_input, sample_output, query_input)
    except Exception as e:
        print(f"An error occurred while running the script: {e}")
        break

    save_to_file(response,save_path)

    # if iter_time > 5:
    #     break

autoGenerating: 100%|██████████| 700/700 [8:57:45<00:00, 46.09s/it]


看一下长度

合成成一个jsonl文件

再召唤GPT老师

已知在python中

save_path = "/content/drive/MyDrive/GPTData/Haruhi-AutoFirst/"

save_name = "/content/drive/MyDrive/GPTData/Haruhi_first_merge.jsonl"

save_path中存储了很多jsonl格式的文件，后缀名是.txt，每行是一个json，其中包含中文。

对save_path中所有的.txt文件进行读取，逐行解析，

如果解析成功，则append到一个list中
如果解析失败，则继续解析下一个文件

最后将list，以jsonl格式存储到save_name中

In [ ]:
import json

save_path = "/content/drive/MyDrive/GPTData/Haruhi-AutoFirst/"
save_name = "/content/drive/MyDrive/GPTData/Haruhi_first_merge.jsonl"

# 定义一个空列表，用于存储解析成功的json数据
json_list = []

# 遍历save_path目录下的所有txt文件
for file_name in os.listdir(save_path):
    if file_name.endswith(".txt"):
        file_path = os.path.join(save_path, file_name)
        with open(file_path, "r", encoding="utf-8") as f:
            # 逐行读取文件内容
            for line in f:
                # 尝试解析读取到的行数据
                try:
                    my_str = line.strip()
                    my_str = my_str.replace("'", "\"")
                    json_data = json.loads(my_str)
                    json_list.append(json_data)
                except:
                    my_str = line.strip()
                    print('warining in line ', my_str)
                    continue

# 将解析成功的json数据以jsonl格式写入save_name文件中
with open(save_name, "w", encoding="utf-8") as f:
    for json_data in json_list:
        f.write(json.dumps(json_data, ensure_ascii=False) + "\n")

In [ ]:
print(len(json_list))

7079


In [ ]:
print(chat_datas[0])
print(chat_datas[1])

{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」', 'keywords': ['Python', '计算机课']}
{'role_A': '阿虚', 'role_B': '春日', 'query': '「你想写一个什么样的程序呢？」', 'response': '「我想写一个能够预测未来的程序，可以预测天气、地震、彩票号码等等。」', 'keywords': ['什么样的程序', '写程序']}


额外再做一个事情吧。把chat_datas做成一个更好的形式。

In [ ]:
with open('/content/temp_all_chat.jsonl','w', encoding="utf-8") as f:
    for chat in chat_datas:
        temp_json1 = {'role':chat['role_A'],'text':chat['query']}
        temp_json2 = {'role':chat['role_B'],'text':chat['response']}
        f.write(json.dumps(temp_json1, ensure_ascii=False) + "\n")
        f.write(json.dumps(temp_json2, ensure_ascii=False) + "\n")